In [1]:
from Datasets.data import AdultDataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import optuna

Total samples: 32561
Number of features: 108
Old data samples: 16280, Positive samples: 3444
New data samples: 16281, Positive samples: 4397


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [3]:
# Load and prepare data
def load_data(dataset_name):
    if dataset_name in ['Adult', 'Bank']:
        dataset = AdultDataset()
        X, y, feature_names, x = dataset.load_data()
        X_old, y_old, X_new, y_new = dataset.split_old_new_data(X, y)
        return X_old, y_old, X_new, y_new, feature_names, x
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")

In [4]:
X_old, y_old, X_new, y_new, feature_names, x = load_data('Adult')
# Verify the split
print(f"Old data shape: {X_old}, Positive samples: {y_old.sum()}")
print(f"New data shape: {X_new}, Positive samples: {y_new.sum()}")

Old data shape: [[4.10000e+01 3.20744e+05 1.30000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [2.50000e+01 2.59688e+05 7.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [1.90000e+01 1.01549e+05 1.00000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 ...
 [5.40000e+01 1.14520e+05 1.00000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [4.00000e+01 3.22140e+04 1.20000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [5.60000e+01 7.62810e+04 1.30000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]], Positive samples: 3441
New data shape: [[3.90000e+01 7.75160e+04 1.30000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [2.80000e+01 3.38409e+05 1.30000e+01 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [3.70000e+01 2.84582e+05 1.40000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 ...
 [4.00000e+01 1.54374e+05 9.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [5.80000e+01 1.51910e+05 9.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [5.20000e+01 2.87927e+05 9.00000e+00 ... 

In [5]:
from Models.models import train_old_model, train_new_models
from Models.lct import LocalCorrectionTree

Old model accuracy: 0.8550
Corrected model accuracy: 0.8800
Number of samples with non-zero corrections: 200
Percentage of samples corrected: 100.00%

Example corrections:
Sample 0:
  Old scores: [0.35111201 0.64888799]
  Correction: [-0.64131982  0.64131982]
  New scores: [-0.29020781  1.2902078 ]
  True label: 1

Sample 1:
  Old scores: [0.13209533 0.86790467]
  Correction: [-0.26724579  0.26724575]
  New scores: [-0.13515045  1.13515041]
  True label: 1

Sample 2:
  Old scores: [0.52459325 0.47540675]
  Correction: [-0.04627001  0.04626997]
  New scores: [0.47832324 0.52167672]
  True label: 1

Sample 3:
  Old scores: [0.14736477 0.85263523]
  Correction: [-0.64131982  0.64131982]
  New scores: [-0.49395504  1.49395504]
  True label: 1

Sample 4:
  Old scores: [0.04450024 0.95549976]
  Correction: [-0.64131982  0.64131982]
  New scores: [-0.59681958  1.59681957]
  True label: 1


Simplified tree structure:
Node: X[5] <= -0.1166
  Node: X[1] <= 0.6981
    Node: X[2] <= -0.4248
      

In [6]:
def evaluate_old_model(dataset, X, y, old_model):
    """
    Evaluate the old model on the given dataset (X, y).
    Returns a dictionary of metrics with lists of values.
    """
    y_pred = old_model.predict(X)
    is_multiclass = dataset in ['CTG', '7-point']
    
    if is_multiclass:
        precision = precision_score(y, y_pred, average='macro', zero_division=0)
        recall = recall_score(y, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y, y_pred, average='macro', zero_division=0)
    else:
        precision = precision_score(y, y_pred, zero_division=0)
        recall = recall_score(y, y_pred, zero_division=0)
        f1 = f1_score(y, y_pred, zero_division=0)
    
    accuracy = accuracy_score(y, y_pred)
    
    return {
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1': [f1]
    }

In [7]:
def evaluate_new_models(dataset, models, X_test, y_test, old_model):
    """
    Evaluate new models including LCT on the test set.
    Returns metrics formatted for error bar plotting.
    """
    metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    is_multiclass = dataset in ['CTG', '7-point']
    old_scores_test = old_model.predict_proba(X_test)
    
    for model_name, model in models.items():
        if model_name == 'Ours':  # LCT model
            corrections = model.predict(X_test)
            corrected_scores = old_scores_test + corrections
            y_pred = np.argmax(corrected_scores, axis=1)
        else:
            if model_name.endswith('+'):
                # Models with old scores as additional features
                X_test_with_scores = np.hstack([X_test, old_scores_test])
                y_pred = model.predict(X_test_with_scores)
            else:
                # Base models
                y_pred = model.predict(X_test)
        
        if is_multiclass:
            precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
            recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
        else:
            precision = precision_score(y_test, y_pred, zero_division=0)
            recall = recall_score(y_test, y_pred, zero_division=0)
            f1 = f1_score(y_test, y_pred, zero_division=0)
        
        accuracy = accuracy_score(y_test, y_pred)
        
        metrics['Accuracy'].append(accuracy)
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['F1'].append(f1)
    
    return metrics


In [8]:
def plot_metrics_comparison(dataset, metrics, model_names):
    """
    Create comparison plots with error bars for model metrics.
    """
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1']
    
    # Define model colors and markers
    colors = {
        'L1-LR': '#1f77b4',
        'L1-LR+': '#1f77b4',
        'L2-LR': '#ff7f0e',
        'L2-LR+': '#ff7f0e',
        'DT': '#2ca02c',
        'DT+': '#2ca02c',
        'RF': '#8c564b',
        'RF+': '#8c564b',
        'LGBM': '#9467bd',
        'LGBM+': '#9467bd',
        'LGBM+C': '#9467bd',
        'Ours': '#d62728',
        'Old': '#7f7f7f'
    }
    
    for idx, metric in enumerate(metrics_names):
        ax = axes[idx]
        y_pos = np.arange(len(model_names))
        values = metrics[metric]
        
        # Plot horizontal lines with error bars
        for i, (name, value) in enumerate(zip(model_names, values)):
            color = colors.get(name, 'black')
            mean_val = np.mean(value)
            std_val = np.std(value)
            
            # Plot horizontal line
            line_length = 0.02
            ax.plot([mean_val-line_length, mean_val+line_length], [i, i],
                   color=color, linewidth=2, solid_capstyle='butt')
            
            # Plot error bars
            if name.endswith('+'):
                marker = 'o'
                fillstyle = 'none'
            else:
                marker = 'o'
                fillstyle = 'full'
            
            ax.errorbar(mean_val, i, xerr=std_val,
                       fmt=marker, color=color, capsize=3,
                       markersize=4, fillstyle=fillstyle,
                       elinewidth=1, capthick=1)
        
        ax.set_yticks(y_pos)
        ax.set_yticklabels(model_names)
        ax.set_title(f'({chr(97+idx)}) {metric} ({dataset})')
        ax.grid(True, alpha=0.3, linestyle='--')
        
        # Set x-axis limits and ticks
        all_values = [np.mean(v) for v in values]
        min_val = min(all_values) - 0.05
        max_val = max(all_values) + 0.05
        ax.set_xlim(min_val, max_val)
        
        # Format x-axis ticks to match the image
        ax.set_xticks(np.linspace(min_val, max_val, 5))
        ax.set_xticklabels([f'{x:.2f}' for x in ax.get_xticks()])
    
    plt.tight_layout()
    return fig

In [9]:
def print_tree(lct, node_id=0, depth=0, feature_names=None):
    """
    Print the Local Correction Tree structure.
    Args:
        lct: LocalCorrectionTree instance
        node_id: Current node ID
        depth: Current depth in tree
        feature_names: Optional list of feature names
    """
    feature_idx, threshold, w_node = lct.nodes[node_id]
    left_id, right_id = lct.children[node_id]
    
    indent = "  " * depth
    if feature_idx == -1:
        print(f"{indent}Leaf: correction = {w_node.round(4)}")
    else:
        feature = f"X[{feature_idx}]" if feature_names is None else feature_names[feature_idx]
        print(f"{indent}Node: {feature} <= {threshold:.4f}")
        print_tree(lct, left_id, depth + 1, feature_names)
        print_tree(lct, right_id, depth + 1, feature_names)

In [10]:
datasets = ['Adult']
for dataset in datasets:
    print(f"\nProcessing {dataset} dataset")
    
    # Define only base models
    base_models = ['L1-LR', 'L2-LR', 'DT', 'RF', 'LGBM']
    
    # 1. Load data
    if dataset == '7-point':
        X_old, y_old, X_new, y_new, X_clinical = load_data(dataset)
    else:
        X_old, y_old, X_new, y_new, feature_names, x = load_data(dataset)
    
    X_new, X_old = np.array(X_new), np.array(X_old)
    y_new, y_old = np.array(y_new), np.array(y_old)
    
    # 2. Train base models on old data
    base_models_dict = train_old_model(dataset, X_new, X_old, y_new, y_old)
    
    # 3. Perform 5-fold cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_metrics = {
        'Accuracy': {model: [] for model in base_models},
        'Precision': {model: [] for model in base_models},
        'Recall': {model: [] for model in base_models},
        'F1': {model: [] for model in base_models}
    }
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_old, y_old)):
        print(f"Processing fold {fold + 1}/5")
        
        X_train, X_test = X_old[train_idx], X_old[test_idx]
        y_train, y_test = y_old[train_idx], y_old[test_idx]
        
        # Evaluate each base model individually
        for model_name in base_models:
            fold_metrics = evaluate_old_model(dataset, X_test, y_test, base_models_dict[model_name])
            
            for metric in all_metrics:
                all_metrics[metric][model_name].extend(fold_metrics[metric])
    
    # 4. Prepare metrics for plotting
    plot_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    for metric in plot_metrics:
        for model in base_models:
            plot_metrics[metric].append(all_metrics[metric][model])
    
    # 5. Create and save plot
    fig = plot_metrics_comparison(dataset, plot_metrics, base_models)
    plt.savefig(f'{dataset}_base_models_comparison.png', bbox_inches='tight', dpi=400)
    plt.close()
    
    # 6. Print numerical results
    print(f"\nBase Model Results for {dataset}:")
    for metric in plot_metrics:
        print(f"\n{metric}:")
        for name, values in zip(base_models, plot_metrics[metric]):
            mean_val = np.mean(values)
            std_val = np.std(values)
            print(f"{name}: {mean_val:.4f} ± {std_val:.4f}")

print("\nDone evaluating base models.")


Processing Adult dataset


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternati

[LightGBM] [Info] Number of positive: 2271, number of negative: 8582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 679
[LightGBM] [Info] Number of data points in the train set: 10853, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 2270, number of negative: 8583
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002435 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 669
[LightGBM] [Info] Number of data points in the train set: 10853, number of used features: 77
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


...

[LightGBM] [Info] Number of positive: 2271, number of negative: 8583
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 673
[LightGBM] [Info] Number of data points in the train set: 10854, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-01-30 00:52:05,764] Trial 99 finished with value: 0.6599992941886489 and parameters: {'learning_rate': 0.006718295950762499, 'min_split_gain': 1.2108669793716555e-05, 'reg_alpha': 7.500303361004378e-06, 'reg_lambda': 0.0016204589545781246, 'n_estimators': 175, 'num_leaves': 49}. Best is trial 73 with value: 0.6948884883574754.
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 4435, number of negative: 11846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002759 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data points in the train set: 16281, number of used features: 86
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.272403 -> initscore=-0.982463
[LightGBM] [Info] Start training from score -0.982463
Processing fold 1/5
Processing fold 2/5
Processing fold 3/5
Processing fold 4/5
Processing fold 5/5


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  war


Base Model Results for Adult:

Accuracy:
L1-LR: 0.8649 ± 0.0036
L2-LR: 0.8653 ± 0.0037
DT: 0.8699 ± 0.0041
RF: 0.8748 ± 0.0026
LGBM: 0.8834 ± 0.0039

Precision:
L1-LR: 0.7111 ± 0.0123
L2-LR: 0.7144 ± 0.0127
DT: 0.7285 ± 0.0158
RF: 0.7668 ± 0.0149
LGBM: 0.7580 ± 0.0091

Recall:
L1-LR: 0.5966 ± 0.0090
L2-LR: 0.5937 ± 0.0102
DT: 0.6033 ± 0.0079
RF: 0.5778 ± 0.0096
LGBM: 0.6500 ± 0.0148

F1:
L1-LR: 0.6488 ± 0.0086
L2-LR: 0.6484 ± 0.0090
DT: 0.6600 ± 0.0086
RF: 0.6589 ± 0.0055
LGBM: 0.6998 ± 0.0114

Done evaluating base models.


In [11]:
datasets = ['Adult']
for dataset in datasets:
    print(f"\nProcessing {dataset} dataset")
    
    base_models = ['L1-LR', 'L2-LR', 'DT', 'RF', 'LGBM']
    enhanced_models = ['L1-LR+', 'L2-LR+', 'DT+', 'RF+', 'LGBM+', 'LGBM+C', 'Ours']
    model_names = base_models + enhanced_models + ['Old']
    
    # 1. Load data
    if dataset == '7-point':
        X_old, y_old, X_new, y_new, X_clinical = load_data(dataset)
    else:
        X_old, y_old, X_new, y_new, feature_names, x = load_data(dataset)
    
    X_new = np.array(X_new)
    y_new = np.array(y_new)
    
    # 2. Train and evaluate old model with multiple runs
    n_runs = 5
    old_metrics_runs = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    for run in range(n_runs):
        old_model = train_old_model(dataset, X_new, y_new)
        run_metrics = evaluate_old_model(dataset, X_new, y_new, old_model)
        for metric in old_metrics_runs:
            old_metrics_runs[metric].append(run_metrics[metric][0])
    
    # 3. Perform 5-fold cross validation for new models
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_metrics = {
        'Accuracy': {model: [] for model in model_names[:-1]},
        'Precision': {model: [] for model in model_names[:-1]},
        'Recall': {model: [] for model in model_names[:-1]},
        'F1': {model: [] for model in model_names[:-1]}
    }
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_new, y_new)):
        print(f"Processing fold {fold + 1}/5")
        
        X_train, X_test = X_new[train_idx], X_new[test_idx]
        y_train, y_test = y_new[train_idx], y_new[test_idx]
        
        # Get old model predictions for enhanced models
        old_scores_train = old_model.predict_proba(X_train)
        old_scores_test = old_model.predict_proba(X_test)
        
        # Train base models
        base_model_dict = train_new_models(dataset, X_train, y_train, old_model, LocalCorrectionTree)
        
        # Train enhanced models with old model scores
        X_train_enhanced = np.hstack([X_train, old_scores_train])
        X_test_enhanced = np.hstack([X_test, old_scores_test])
        enhanced_model_dict = train_new_models(dataset, X_train_enhanced, y_train, old_model, LocalCorrectionTree)
        
        # Combine all models
        fold_models = {**base_model_dict, **enhanced_model_dict}
        
        # Evaluate models
        fold_metrics = evaluate_new_models(dataset, fold_models, X_test, y_test, old_model)
        
        for metric in all_metrics:
            for model in model_names[:-1]:
                if model in base_models:
                    all_metrics[metric][model].append(fold_metrics[metric][model])
                else:
                    # For enhanced models, use enhanced data
                    if model == 'Ours':
                        # LCT uses original features but adds corrections
                        all_metrics[metric][model].append(fold_metrics[metric][model])
                    else:
                        all_metrics[metric][model].append(fold_metrics[metric][model])
                        
    print("\nFinal Local Correction Tree Structure:")
    if 'Ours' in enhanced_models:  # Check if LCT model exists
        enhanced_models['Ours'].simplify()  # Remove redundant nodes
        print_tree(enhanced_models['Ours'], feature_names=feature_names)
    
    # 4. Prepare metrics for plotting
    plot_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    for metric in plot_metrics:
        plot_metrics[metric].append(old_metrics_runs[metric])
        for model in model_names[:-1]:
            plot_metrics[metric].append(all_metrics[metric][model])
    
    # 5. Create and save plot
    fig = plot_metrics_comparison(dataset, plot_metrics, model_names)
    plt.savefig(f'{dataset}_model_comparison.png', bbox_inches='tight', dpi=300)
    plt.close()
    
    # 6. Print numerical results
    print(f"\nResults for {dataset}:")
    for metric in plot_metrics:
        print(f"\n{metric}:")
        for name, values in zip(model_names, plot_metrics[metric]):
            mean_val = np.mean(values)
            std_val = np.std(values)
            print(f"{name}: {mean_val:.4f} ± {std_val:.4f}")

print("\nDone evaluating old model and new models.")


Processing Adult dataset


TypeError: train_old_model() missing 2 required positional arguments: 'y_new' and 'y_old'